<a href="https://colab.research.google.com/github/adriansletten/CV_test/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CV test
This file contains the code used for finetuning a yolo-v8 model on the lions and hippos dataset.


## step 1: Install required packages


In [1]:
!pip install ultralytics==8.0.196
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.1/631.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstal

## Step 2: Load dataset from roboflow

In [2]:
from google.colab import userdata
api_key = userdata.get('roboflow_api_key')

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key=api_key)
project = rf.workspace("adriansletten").project("lions_and_hippos")
dataset = project.version(1).download("yolov8")



loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to lions_and_hippos-1 in yolov8:: 100%|██████████| 580/580 [00:00<00:00, 2390.14it/s]


In [8]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data={dataset.location}/data.yaml \
epochs=200 \
imgsz=640


100% 21.5M/21.5M [00:00<00:00, 171MB/s]
New https://pypi.org/project/ultralytics/8.0.220 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/lions_and_hippos-1/data.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False,

In [9]:
!yolo task=detect \
mode=val \
model=/content/runs/detect/train/weights/best.pt \
data={dataset.location}/data.yaml


Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/lions_and_hippos-1/valid/labels.cache... 55 images, 1 backgrounds, 0 corrupt: 100% 55/55 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 4/4 [00:03<00:00,  1.21it/s]
                   all         55         54      0.759      0.821      0.828      0.625
          hippopotamus         55         40      0.947          1      0.993      0.905
                  lion         55         14      0.571      0.643      0.664      0.345
Speed: 5.2ms preprocess, 27.0ms inference, 0.0ms loss, 5.3ms postprocess per image
Results saved to runs/detect/val
💡 Learn more at https://docs.ultralytics.com/modes/val


In [10]:
!yolo task=detect \
mode=predict \
model=/content/runs/detect/train/weights/best.pt \
conf=0.25 \
source={dataset.location}/test/images

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/29 /content/lions_and_hippos-1/test/images/hippopotamus-102_jpg.rf.ead18b78e716c5b216f29ee88231df48.jpg: 416x640 1 hippopotamus, 96.4ms
image 2/29 /content/lions_and_hippos-1/test/images/hippopotamus-103_jpg.rf.2a9472670c0a28c93dd7d6d0e0d4dfea.jpg: 640x640 1 hippopotamus, 16.9ms
image 3/29 /content/lions_and_hippos-1/test/images/hippopotamus-110_jpg.rf.d051ca2b8b0610206634a719e814c3c8.jpg: 448x640 1 hippopotamus, 61.0ms
image 4/29 /content/lions_and_hippos-1/test/images/hippopotamus-115_jpg.rf.cbe1155b9fb4135393ad9a004355f7d5.jpg: 480x640 1 hippopotamus, 61.2ms
image 5/29 /content/lions_and_hippos-1/test/images/hippopotamus-124_jpg.rf.b652e4622f4140fe079eca7f9e25b4f1.jpg: 640x448 1 hippopotamus, 65.7ms
image 6/29 /content/lions_and_hippos-1/test/images/hippopotamus-128_jpg.rf.39c576759adbead2fe22bcde30fca56c.jp

In [11]:
project.version(dataset.version).deploy(model_type="yolov8", model_path=f"/content/runs/detect/train/")

View the status of your deployment at: https://app.roboflow.com/adriansletten/lions_and_hippos/1
Share your model with the world at: https://universe.roboflow.com/adriansletten/lions_and_hippos/model/1


In [12]:
!zip /content/runs.zip /content/runs

  adding: content/runs/ (stored 0%)
